In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import mysql.connector
import plotly
import plotly.plotly as py #library https://plot.ly/pandas/
import plotly.graph_objs as go
#import cufflinks as cf

plotly.tools.set_credentials_file(username='zhenxu66', api_key='yZ5rrla4FGut06wbewMo')

In [2]:
df_allnd100list= pd.read_csv('./csv/list.csv')

In [4]:
df_allnd100list.head(2)

,id,symbol,url,descShort,descLong,CEO,address,shares_out(million),inst_own,mark_cap(billion)
0,1,AAL,http://www.cnbc.com/quotes/?symbol=AAL,American Airlines Group Inc. is a holding comp...,American Airlines Group Inc. is a holding comp...,William Douglas Parker,"4333 Amon Carter Blvd, Fort Worth, Texas, Unit...",460.5,0.8186,14.2
1,2,COST,http://www.cnbc.com/quotes/?symbol=COST,Costco Wholesale Corporation is engaged in the...,Costco Wholesale Corporation is engaged in the...,W. Craig Jelinek,"999 Lake Dr, Issaquah, Washington, United Stat...",438.2,0.7361,99.4


In [5]:
#my_string = "4333 Amon Carter Blvd, Fort Worth, Texas, United States "
#print([x.strip() for x in my_string.split(',')][2])
def get_state(address):
    return [x.strip() for x in address.split(',')][2]

def get_city(address):
    return [x.strip() for x in address.split(',')][1]

Texas


In [6]:
df_allnd100list['state'] = df_allnd100list['address'].apply(get_state)
df_allnd100list['city'] = df_allnd100list['address'].apply(get_city)

In [7]:
df_allnd100list.head(5)

,id,symbol,url,descShort,descLong,CEO,address,shares_out(million),inst_own,mark_cap(billion),state,city
0,1,AAL,http://www.cnbc.com/quotes/?symbol=AAL,American Airlines Group Inc. is a holding comp...,American Airlines Group Inc. is a holding comp...,William Douglas Parker,"4333 Amon Carter Blvd, Fort Worth, Texas, Unit...",460.5,0.8186,14.2,Texas,Fort Worth
1,2,COST,http://www.cnbc.com/quotes/?symbol=COST,Costco Wholesale Corporation is engaged in the...,Costco Wholesale Corporation is engaged in the...,W. Craig Jelinek,"999 Lake Dr, Issaquah, Washington, United Stat...",438.2,0.7361,99.4,Washington,Issaquah
2,3,CTSH,http://www.cnbc.com/quotes/?symbol=CTSH,Cognizant Technology Solutions Corporation is ...,Cognizant Technology Solutions Corporation is ...,Francisco D'Souza,"500 Frank W Burr Blvd, Teaneck, New Jersey, Un...",580.0,0.9236,41.9,New Jersey,Teaneck
3,4,EBAY,http://www.cnbc.com/quotes/?symbol=EBAY,"eBay Inc. (eBay) is a commerce company, which ...","eBay Inc. (eBay) is a commerce company, which ...",Devin Norse Wenig,"2025 Hamilton Ave, San Jose, California, Unite...",989.5,0.8723,31.4,California,San Jose
4,5,EA,http://www.cnbc.com/quotes/?symbol=EA,"Electronic Arts Inc. develops, markets, publis...","Electronic Arts Inc. develops, markets, publis...",Andrew Wilson,"209 Redwood Shores Pkwy, Redwood City, Califor...",304.8,0.9769,32.3,California,Redwood City


In [8]:
data_state = [go.Histogram(x=df_allnd100list['state'])]

In [9]:
py.iplot(data_state, filename='simple-histogram')

In [10]:
df_state = df_allnd100list[df_allnd100list['state']=='California']

In [11]:
def city_merge_company(city):
    df_state_city = df_state[df_state['city']==city]
    return df_state_city['symbol'].str.cat(sep=',')
    
#city_merge_company('Mountain View')

In [12]:
data_city = [go.Histogram(x=df_state.city)]

In [13]:
py.iplot(data_city, filename='simple-histogram')

In [14]:
value_counts = df_state['city'].value_counts()
df_city = value_counts.rename_axis('city').reset_index(name='counts')
df_city['symbols'] = df_city['city'].apply(city_merge_company)
print(df_city)

             city  counts                                           symbols
0        San Jose      10  EBAY,CSCO,XLNX,WDC,PYPL,MXIM,CDNS,AVGO,ADBE,ALGN
1   Mountain View       5                         SNPS,SYMC,INTU,GOOG,GOOGL
2     Santa Clara       3                                    NVDA,INTC,AMAT
3       San Diego       2                                         QCOM,ILMN
4   Thousand Oaks       1                                              AMGN
5          Dublin       1                                              ROST
6       Palo Alto       1                                              TSLA
7          Novato       1                                              BMRN
8    Santa Monica       1                                              ATVI
9        Milpitas       1                                              KLAC
10     Pleasanton       1                                              WDAY
11   Redwood City       1                                                EA
12      Cupe

In [15]:
#df_city['symbols'].tolist()

In [16]:
trace = go.Pie(labels=df_city['city'], values=df_city['counts'], text = df_city.symbols, textinfo='label+value')
py.iplot([trace],filename='basic_pie_chart')

In [17]:
fig_state = {
    'data': [
        {
            'x': df_state['mark_cap(billion)'], 
            'y': df_state['shares_out(million)'], 
            'text': df_state.symbol + ' City:'+ df_state.city, 
            'orientation': 'h',
            'mode': 'markers', #+text  markers
            'marker':dict(
                size= 8, #30*df_allnd100list['inst_own'],
                color = df_state['inst_own'], #set color equal to a variable
                colorscale='Viridis',
                showscale=True
            ),
            'name': 'Results'},      

    ],
    'layout': {
        'title': 'inst_own compared with shares_out (million) and mark_cap (billion)',
        'xaxis': {'title': 'mark_cap(billion)', 'type': 'log'},
        'yaxis': {'title': "shares_out(million)"},
        'showlegend': True,
        'annotations':[ #not working
            dict(
                x=1100,#df_allnd100list[df_allnd100list.symbol=='AAPL']['mark_cap(billion)'].item(),
                y=4800,#df_allnd100list[df_allnd100list.symbol=='AAPL']['shares_out(million)'].item(),
                xref='x',
                yref='y',
                text='dict Text is here',
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
            ),
        ]
    }
}

In [18]:
py.iplot(fig_state, filename='scatter') 

In [41]:
trace1 = go.Histogram(
    x=df_state['mark_cap(billion)'],
    #histnorm='percent',
    name='california',
    xbins=dict(
        start=10.0,
        end=100.0,
        size=10
    ),
    marker=dict(
        color='#0000ff',
    ),
    opacity=0.75
    #text = df_state['symbol']
)
trace2 = go.Histogram(
    x=df_allnd100list['mark_cap(billion)'],
    name='all',
    xbins=dict(
        start=10.0,
        end=100,
        size=10
    ),
    marker=dict(
        color='#EB89B5'
    ),
    opacity=0.75
    #text = df_allnd100list['symbol']
)
data = [trace1, trace2]

layout = go.Layout(
    title='mark_cap(billion) comparison of California with all US (NASDAQ100)',
    xaxis=dict(
        title='mark_cap(billion)'
    ),
    yaxis=dict(
        title='Count'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styled histogram')